<a href="https://colab.research.google.com/github/cr2uchile/Quality_Control_FireScarCL/blob/main/1_generate_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

import os
import glob
import csv
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from osgeo import gdal
# OBS printGetHistogram(0,2,2, approx_ok=0 ) !! Without change gives wrong values for large rasters

!pip install simpledbf
from simpledbf import Dbf5

!pip install pyshp
import shapefile

  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13801 sha256=c4409d4cead75c539e39205506e8a56e01b35a09568cfdd9f9de71252311eb57
  Stored in directory: /root/.cache/pip/wheels/24/43/f4/39ad84349e5358346be977fe626160f5625fdd3ea8e017518c
Successfully built simpledbf
     |████████████████████████████████| 219 kB 4.3 MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37325 sha256=94a97c5424b329d90cb109e9a1e1265922cc84f0074cfe9033082f6a9ea39eda
  Stored in directory: /root/.cache/pip/wheels/43/f8/87/53c8cd41545ba20e536ea29a8fcb5431b5f477ca50d5dffbbe
Successfully built pyshp


In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 
#listOfFiles = os.listdir('drive/MyDrive')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:

sys.path.insert(0,'/content/drive/Shareddrives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/ColabNotebooks/')
from common import *
print (output_path)


drive/Shared drives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Report/


### Save Images

In [ ]:
cmapScar = ListedColormap(['k', 'w'], N=2)
cmapSev = ListedColormap(['k', 'c', 'm', 'y'], N=4)
# REF: https://matplotlib.org/stable/tutorials/colors/colors.html

def save_images(filePath, dirName, fireRegion, fireId, fireThreshold, fireDate, fileExt):

  print('image for ',fireId)
  fileName=os.path.basename(filePath)
  (fileNameOnly, fileExt)=os.path.splitext(fileName) 

  t1 = gdal.Open(filePath)
  #print(dirName+'/Severity_'+fireRegion+'_'+fireId+'_'+ fireThreshold+'_'+ fireDate+fileExt)
  t2 = gdal.Open(dirName+'/Severity_'+fireRegion+'_'+fireId+'_'+ fireThreshold+'_'+ fireDate+fileExt)
  #print(dirName+'/RdNBR'+fireRegion+'_'+fireId+'_'+ fireThreshold+'_'+ fireDate+fileExt)
  t3 = gdal.Open(dirName+'/RdNBR_'+fireRegion+'_'+fireId+'_'+ fireThreshold+'_'+ fireDate+fileExt)

  b1 =  t1.GetRasterBand(1) # channel
  b2 =  t2.GetRasterBand(1) # channel
  b3 =  t3.GetRasterBand(1) # channel        
  a1 =  b1.ReadAsArray()
  a2 =  b2.ReadAsArray()
  a3 =  b3.ReadAsArray() 
  fig=plt.figure()
  
  ax1 = plt.subplot(311)
  im1 = ax1.imshow(a1, cmap=cmapScar)
  fig.colorbar(im1)
  #plt.colorbar()
  ax2 = plt.subplot(312)
  im2 = ax2.imshow(a2, cmap=cmapSev)
  fig.colorbar(im2)
  #plt.colorbar()
  ax3 = plt.subplot(313)
  im3 = ax3.imshow(a3, cmap='terrain')
  fig.colorbar(im3)
  #plt.colorbar()
  #plt.show()
  plt.savefig(output_path+'images/'+fileNameOnly+'_tif.png')    


### get Fire Details

In [ ]:
def get_shapeDetails(fireId, shape_path):
  print (fireId)            
  #shape_path = listFires[fireId, 'FireScar', '.shp']
  sf = shapefile.Reader(shape_path)
  print(sf.bbox)
  

In [ ]:
def get_fireDetails(listFires, fireId):
      if idFilter and fireId != idFilter:
          return ''
      print (fireId)      
      tif1Path = listFires[fireId, 'FireScar', '.tif']
      tif2Path = listFires[fireId, 'Severity', '.tif']
      tif3Path = listFires[fireId, 'RdNBR', '.tif']
      tif4Path = listFires[fireId, 'ImgPreF', '.tif']
      tif5Path = listFires[fireId, 'ImgPosF', '.tif']
      shape1Path = listFires[fireId, 'FireScar', '.dbf']
      shape2Path = listFires[fireId, 'Severity', '.dbf']
      
      # Check all basic asserts between tif1,2,3 and tif4,5, layers, etc
      t1 = gdal.Open(tif1Path)
      t2 = gdal.Open(tif2Path)
      t3 = gdal.Open(tif3Path)
      t4 = gdal.Open(tif4Path)
      t5 = gdal.Open(tif5Path)
      tif1Meta = t1.GetMetadata()
      assert (tif1Meta == t2.GetMetadata() == t3.GetMetadata() == t4.GetMetadata() == t5.GetMetadata()), "Same Type= Area definition"
      tif1Proj = t1.GetProjection()
      assert (tif1Proj == t2.GetProjection() == t3.GetProjection() == t4.GetProjection() == t5.GetProjection()), "Same Projection definition"

      tif1Geo = t1.GetGeoTransform()
      tif2Geo = t2.GetGeoTransform()
      tif3Geo = t3.GetGeoTransform()
          # REF:adfGeoTransform[0] /* top left x */
          # adfGeoTransform[1] /* w-e pixel resolution */
          # adfGeoTransform[3] /* top left y */
          # adfGeoTransform[5] /* n-s pixel resolution (negative value) */
      tif4Geo = t4.GetGeoTransform()
      tif5Geo = t5.GetGeoTransform()

      tif1RCount = t1.RasterCount
      assert (tif1RCount == t2.RasterCount == t3.RasterCount), "Wrong Raster count values"
      tif4RCount = t4.RasterCount
      assert (tif4RCount == t5.RasterCount), "Wrong Raster count values"

      b1 =  t1.GetRasterBand(1) # channel
      b2 =  t2.GetRasterBand(1) # channel
      b3 =  t3.GetRasterBand(1) # channel

      assert not b1.GetNoDataValue(), "No empty values"
      assert b1.GetNoDataValue() == b2.GetNoDataValue() == b3.GetNoDataValue(), "No empty values"
      tif1X = t1.RasterXSize
      tif1Y = t1.RasterYSize
      tif2X = t2.RasterXSize
      tif2Y = t2.RasterYSize
      tif3X = t3.RasterXSize
      tif3Y = t3.RasterYSize
      tif4X = t4.RasterXSize
      tif4Y = t4.RasterYSize
      tif5X = t5.RasterXSize
      tif5Y = t5.RasterYSize

      (tif1Min, tif1Max) = b1.ComputeRasterMinMax()
      (tif2Min, tif2Max) = b2.ComputeRasterMinMax()
      (tif1Zeros, tif1Ones) = b1.GetHistogram(0,2,2, approx_ok=0) # HAs 0 and 1          
      (tif2Zeros, tif2Ones, tif2Twos, tif2Threes) = b2.GetHistogram(0,4,4, approx_ok=0) # Has 0 1 2 3 
      (tif3Min, tif3Max) = b3.ComputeRasterMinMax() # does not have zero values, only float range between X and Y

      #dbf1 = Dbf5(shape1Path)
      dbf1 = shapefile.Reader(shape1Path)

      #dbf2 = Dbf5(shape2Path)
      dbf2 = shapefile.Reader(shape2Path)

      row = [fireId, 
                tif1Meta, tif1Proj, tif1Geo[0], tif1Geo[3], tif1Geo[1], tif1Geo[5], tif1RCount, tif1X, tif1Y, tif1Zeros, tif1Ones, tif1Min, tif1Max,
                tif2Geo[0], tif2Geo[3], tif2Geo[1], tif2Geo[5], tif2X, tif2Y, tif2Zeros, tif2Ones, tif2Twos, tif2Threes, tif2Min, tif2Max,
                tif3Geo[0], tif3Geo[3], tif3Geo[1], tif3Geo[5], tif3X, tif3Y, tif3Min, tif3Max, 
                tif4Geo[0], tif4Geo[3], tif4Geo[1], tif4Geo[5], tif4RCount, tif4X, tif4Y,
                tif5Geo[0], tif5Geo[3], tif5Geo[1], tif5Geo[5], tif5X, tif5Y,
                list(dbf1.fields), len(dbf1.fields), len(dbf1.records()), list(dbf2.fields), len(dbf2.fields), len(dbf2.records())]
      row.extend(dbf1.record(0))
      row.extend(dbf2.record(0))
      #row.extend(s2.iloc[0])
      return row

In [ ]:
# delete xml files
def remove_xml_file(fileExt, filePath):
  if fileExt=='.xml':
    os.remove(filePath)
    print(filePath)
    print('removed')

## Traverse regions

In [ ]:
def traverse_regions(folderFilter, subFolderFilter, yearFilter, idFilter):

  if flag_summary:
    summary_filename = 'summary_20211116_VF2.csv'
    summary_file =  open(output_path+summary_filename+regionFilter, mode='w')
    summary_writer = csv.writer(summary_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    # Generate size per file in spreadsheet with right columns:
    columnNames = ['folderRegion','folderR_0', 'folderR_1', 'folderR_2','folderSeason','folderS_0','folderS_1','folderS_2', 'folderS_3', 'fileName','fileExt', 'fireType', 'fireRegion', 'fireId','fireThreshold','fireDate','fileSize']
    if flag_modifDate:
      columnNames.extend(['fileModifDate'])
    summary_writer.writerow(columnNames)

  contentDetails_filename = 'report_content_20211116_VF2.csv'
  if flag_details: # writes a file with column names
    with open(output_path+contentDetails_filename+'columnNames', mode='w') as details_file:
      details_writer = csv.writer(details_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      columnNames=['fireId','tif1Meta','tif1Proj','tif1Geo0', 'tif1Geo3', 'tif1Geo1', 'tif1Geo5', 'tif1RCount', 'tif1X', 'tif1Y', 'tif1Zeros', 'tif1Ones', 'tif1Min', 'tif1Max',
      'tif2Geo0', 'tif2Geo3', 'tif2Geo1', 'tif2Geo5', 'tif2X', 'tif2Y', 'tif2Zeros', 'tif2Ones', 'tif2Twos', 'tif2Threes', 'tif2Min', 'tif2Max',
      'tif3Geo0', 'tif3Geo3', 'tif3Geo1', 'tif3Geo5', 'tif3X', 'tif3Y', 'tif3Min', 'tif3Max', 
      'tif4Geo0', 'tif4Geo3', 'tif4Geo1', 'tif4Geo5', 'tif4RCount', 'tif4X', 'tif4Y',
      'tif5Geo0', 'tif5Geo3', 'tif5Geo1', 'tif5Geo5', 'tif5X', 'tif5Y',
      'columnsShape1', 'lencolumnsShape1', 'lenShape1', 'columnsShape2', 'lencolumnsShape2', 'lenShape2',
      'NIR_min', 'AreaTotal', 'Temporada', 'AreaINC_ha', 'AreaUnchS', 'AreaLowS', 'AreaHighS','Control','AreaMedS', 'Inicio', 'Region', 'NDVI_mean', 'Nombre_Inc', 'Area_Poly', 'KEY', 
                 'AreaTotal', 'Temporada', 'AreaINC_ha', 'Severity', 'AreaUnchS', 'AreaLowS', 'AreaHighS', 'Control', 'AreaMedS', 'Inicio', 'Region', 'Nombre_Inc', 'KEY']
      details_writer.writerow(columnNames)

  # traverse all regions    
  listFolders = filter(os.path.isdir, glob.glob(base_path + folderFilter + '*') )  
  for folderPath in listFolders:  
    print(folderPath)  
    # Filter regions - years that match filters
    listSubfolders = filter(os.path.isdir, glob.glob(folderPath + '/'+subFolderFilter + '*'+yearFilter) )  
    folderRegion=os.path.basename(folderPath)   

    folderRParts=folderRegion.split("_")
    print(folderPath + '/'+folderFilter + '*')
    if flag_details:
        details_file = open(output_path+contentDetails_filename+folderRegion+'.csv', mode='w')
        details_writer = csv.writer(details_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)        
    # traverse all region-season    
    for subfolderPath in listSubfolders:   
      listFiles = filter(os.path.isfile, glob.glob(subfolderPath + '/*') )
      folderSeason=os.path.basename(subfolderPath)            
      print(folderSeason)


      folderSParts=folderSeason.split("_")
      if flag_details:
        listFires = {} # For traversing Fires on types and extensions     
      # traverse all FILES in a region-season  
      for filePath in listFiles:   
        # Ex: FireScar_CL-BI_ID151372_u390_20101211
        
        fileName=os.path.basename(filePath)
        dirName = os.path.dirname(filePath) # for print only
        print(fileName)
        (fileNameOnly, fileExt)=os.path.splitext(fileName) 
        (fireType, fireRegion, fireId, fireThreshold, fireDate)=fileNameOnly.split("_")

        if fileExt == '.xml':
          remove_xml_file(fileExt,filePath)
          continue

        if fireType=='FireScar' and fileExt == '.tif' and flag_printTif == True:
          save_images(filePath, dirName, fireRegion, fireId, fireThreshold, fireDate, fileExt)

        if flag_summary:
          fileSize=os.stat(filePath).st_size
          row = [folderRegion, folderRParts[0], folderRParts[1], folderRParts[2], folderSeason, folderSParts[0], folderSParts[1], folderSParts[2], folderSParts[3], fileName, fileExt,  fireType, fireRegion, fireId, fireThreshold, fireDate, fileSize]
          if flag_modifDate:
            modTimesinceEpoc = os.path.getmtime(filePath)
            fileModifDate= time.strftime('%Y%m%d', time.localtime(modTimesinceEpoc))
            row.extend([fileModifDate])
          print(row)
          summary_writer.writerow(row)

        if flag_details: 
          listFires[fireId, fireType, fileExt]=filePath

      # finishes traversing all files in A region-season
      ##################################################  
      if flag_details: 
        # Now traverses the saved dictionary, all FireIDS (only for getting the DETAILS)
        fireIds = set([ k[0] for k in listFires.keys()])    
        for fireId in fireIds:
          #print (fireId)
          row_details=get_fireDetails(listFires, fireId)
          print(row_details)
          # print(row) BASE ! + details
          if row_details:
            details_writer.writerow(row_details)

    if flag_details:
      details_file.close()  # Closes Detailes Region Files

  if flag_summary:
    summary_file.close() # Closes all regions Summary File

## Initial Setup


In [ ]:
flag_summary = 1
flag_printTif = 0
flag_modifDate = 0
flag_details = 0

regionFilter = '_CL-ML'
regionFilter=''

folderFilter='Fire'+regionFilter
subFolderFilter = 'FireScar'+regionFilter
yearFilter = '2017'

yearFilter = ''
#idFilter='ID38899'
idFilter=''
#idFilter='ID189275'
#print(folderFilter)
traverse_regions('Fire'+regionFilter, 'FireScar'+regionFilter, yearFilter, idFilter)

#traverse_regions('Fire'+'_CL-ML', 'FireScar'+'_CL-ML', yearFilter, idFilter)
#traverse_regions('Fire'+'_CL-VS', 'FireScar'+'_CL-VS', yearFilter, idFilter)



Streaming output truncated to the last 5000 lines.
['Fire_CL-LI_Ohiggins', 'Fire', 'CL-LI', 'Ohiggins', 'FireScar_CL-LI_Ohiggins_2013', 'FireScar', 'CL-LI', 'Ohiggins', '2013', 'RdNBR_CL-LI_ID162832_u560_20130109.tif', '.tif', 'RdNBR', 'CL-LI', 'ID162832', 'u560', '20130109', 12764]
Severity_CL-LI_ID162832_u560_20130109.tif
['Fire_CL-LI_Ohiggins', 'Fire', 'CL-LI', 'Ohiggins', 'FireScar_CL-LI_Ohiggins_2013', 'FireScar', 'CL-LI', 'Ohiggins', '2013', 'Severity_CL-LI_ID162832_u560_20130109.tif', '.tif', 'Severity', 'CL-LI', 'ID162832', 'u560', '20130109', 2381]
ImgPreF_CL-LI_ID162832_u560_20130109.tif
['Fire_CL-LI_Ohiggins', 'Fire', 'CL-LI', 'Ohiggins', 'FireScar_CL-LI_Ohiggins_2013', 'FireScar', 'CL-LI', 'Ohiggins', '2013', 'ImgPreF_CL-LI_ID162832_u560_20130109.tif', '.tif', 'ImgPreF', 'CL-LI', 'ID162832', 'u560', '20130109', 372849]
Severity_CL-LI_ID162832_u560_20130109.prj
['Fire_CL-LI_Ohiggins', 'Fire', 'CL-LI', 'Ohiggins', 'FireScar_CL-LI_Ohiggins_2013', 'FireScar', 'CL-LI', 'Ohiggins